In [ ]:
# imports
import os
import json
import pandas as pd
%pip install SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader
%pip install datasets
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
# soccernet downloads

mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="./soccernet")

mySoccerNetDownloader.downloadGames(files=["Labels-v2.json"], split=["train", "valid", "test"]) # labels
# mySoccerNetDownloader.downloadDataTask(task="spotting-2023", split=["train", "valid", "test", "challenge"]) # highlight labels for 2023 challenge task
# mySoccerNetDownloader.downloadGames(files=["1_ResNET_TF2.npy", "2_ResNET_TF2.npy"], split=["train", "valid", "test"]) # visual features (not needed)

# mySoccerNetDownloader.password = "Password from NDA" # TODO: we need access
# videos:
# mySoccerNetDownloader.downloadGames(files=["1_224p.mkv", "2_224p.mkv"], split=["train", "valid", "test"]) # 224p resolution
# mySoccerNetDownloader.downloadGames(files=["1_720p.mkv", "2_720p.mkv"], split=["train", "valid", "test"]) # 720p resolution

In [17]:
"""
    helper function to convert position (milliseconds) to seconds and extract the half from gameTime

    args:
        position (str): the position in milliseconds as a string
        game_time (str): the gameTime string in the format "1 - mm:ss" or "2 - mm:ss"

    returns:
        tuple: a tuple containing:
            - time_in_seconds (float): the position converted to seconds
            - half (int): the half of the game (1 or 2)
"""
def position_to_seconds_and_half(position, game_time):
    time_in_seconds = float(position) / 1000
    half = int(game_time.split(" - ")[0]) # extract the half from gameTime
    return time_in_seconds, half

In [18]:
# initialize list to hold all data
all_data = []

for root, dirs, files in os.walk('soccernet'):
    for file in files:
        if file == "Labels-v2.json":
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                labels_data = json.load(f)

            match_name = labels_data.get("UrlLocal", "")  # match_name is from the UrlLocal key
            annotations = labels_data.get("annotations", []) # events from the "annotations" key

            for label_entry in annotations:
                # convert position value from milliseconds to seconds
                time_in_seconds, half = position_to_seconds_and_half(label_entry["position"], label_entry["gameTime"])
                match_name_with_half = f"{match_name}{half}" # match transcript "game" name
                label = label_entry["label"]
                all_data.append({
                    "time_in_seconds": time_in_seconds,
                    "match_name": match_name_with_half,
                    "label": label
                })

# convert the list to dataframe and save to csv
labels_df = pd.DataFrame(all_data)
labels_df.to_csv("labels_df.csv", index=False)
print(labels_df.head())

   time_in_seconds                                         match_name  \
0            0.000  england_epl/2014-2015/2015-02-21 - 18-00 Chels...   
1          133.295  england_epl/2014-2015/2015-02-21 - 18-00 Chels...   
2          149.168  england_epl/2014-2015/2015-02-21 - 18-00 Chels...   
3          156.098  england_epl/2014-2015/2015-02-21 - 18-00 Chels...   
4          182.775  england_epl/2014-2015/2015-02-21 - 18-00 Chels...   

              label  
0          Kick-off  
1  Ball out of play  
2          Throw-in  
3  Ball out of play  
4            Corner  


In [19]:
# load transcript dataset
transcript_dataset = load_dataset("SoccerNet/SN-echoes", 
                      name="whisper_v3", 
                      split="en") # load only the English split
transcripts_df = pd.DataFrame(transcript_dataset)

In [20]:
transcripts_df['label'] = 'non-highlight'

# update label based on time and game matching conditions
for idx, row in tqdm(transcripts_df.iterrows(), total=len(transcripts_df), desc="Processing rows"):
    matching_label = labels_df[
        (labels_df['time_in_seconds'] >= row['start_time']) &
        (labels_df['time_in_seconds'] <= row['end_time']) &
        (labels_df['match_name'] == row['game'])
    ]
    if not matching_label.empty: # found a match
        transcripts_df.at[idx, 'label'] = matching_label.iloc[0]['label']

transcripts_df.to_csv("transcripts_df.csv", index=False)
print(transcripts_df.head())

Processing rows: 100%|██████████| 679738/679738 [1:26:44<00:00, 130.62it/s]


   segment_index  start_time    end_time  \
0              0   30.000000   59.980000   
1              1   60.000000   72.440002   
2              2   73.260002   89.199997   
3              3   90.000000  106.180000   
4              4  106.180000  117.360001   

                                                text  \
0                         and everything is possible   
1  Felipe Luis for Azpilicueta and Zouma for Keji...   
2  César opened towards Eden Hazard, Hazard who w...   
3  Cuadrado left again for Hazard, there is Juan ...   
4  and there are quite a few, not only this seaso...   

                                                game          label  
0  england_epl/2014-2015/2015-02-21 - 18-00 Chels...  non-highlight  
1  england_epl/2014-2015/2015-02-21 - 18-00 Chels...  non-highlight  
2  england_epl/2014-2015/2015-02-21 - 18-00 Chels...  non-highlight  
3  england_epl/2014-2015/2015-02-21 - 18-00 Chels...  non-highlight  
4  england_epl/2014-2015/2015-02-21 - 18-00 Chel

In [ ]:
# data analysis
label_counts = labels_df['label'].value_counts()
print(label_counts)
transcripts_label_counts = transcripts_df['label'].value_counts()
print(transcripts_label_counts)

# TODO: we are losing a lot of highlights, we need to investigate why

label
Ball out of play      31810
Throw-in              18918
Foul                  11674
Indirect free-kick    10521
Clearance              7896
Shots on target        5820
Shots off target       5256
Corner                 4836
Substitution           2839
Kick-off               2566
Direct free-kick       2200
Offside                2098
Yellow card            2047
Goal                   1703
Penalty                 173
Red card                 55
Yellow->red card         46
Name: count, dtype: int64
label
non-highlight         624183
Ball out of play       15204
Throw-in                9108
Foul                    6162
Indirect free-kick      5456
Clearance               4208
Shots on target         3124
Shots off target        2771
Corner                  2549
Substitution            1509
Kick-off                1469
Direct free-kick        1207
Offside                 1164
Yellow card             1093
Goal                     425
Penalty                   53
Red card              